#  Setup: Display options

In [ ]:

#  Setting display options 

import pandas as pd
   #
pd.set_option("display.width", 480)

#  Sets horizontal scroll for wide outputs
#
from IPython.display import display, HTML
display(HTML(""))

from tabulate import tabulate

print("--")


# Setup: Connectivity

In [ ]:

from katana import remote

my_client = remote.Client()

print(my_client)


In [ ]:

NUM_PARTITIONS  = 3
   #
DB_NAME         = "my_db"
GRAPH_NAME      = "my_graph"

print("--")


In [ ]:

#  DELETE ALL GRAPHS

for l_database in my_client.databases():
   for l_graph in my_client.get_database(name=l_database.name).graphs_in_database():
      l_handle=my_client.get_database(name=l_database.name).get_graph_by_id(id=l_graph.graph_id)
      l_handle.delete()

for l_graph in my_client.graphs():
   print("GRAPH ID: ", l_graph.graph_id, "      GRAPH Version: ", l_graph.version)

print("--")


In [ ]:

#  DELETE ALL DATABASES

for l_database in my_client.databases():
   if (l_database.name != "default"):
      my_client.get_database(name=l_database.name).delete_database()
      print("--")

for l_database in my_client.databases():
   print("DB ID: ", l_database.database_id, "     DB Name: ", l_database.name)


In [ ]:

#  CREATE DATABASE

my_database = my_client.create_database(name=DB_NAME)

print(my_database.database_id)


In [ ]:

#  CREATE GRAPH
#
my_graph = my_client.get_database(name=DB_NAME).create_graph(name=GRAPH_NAME, num_partitions=NUM_PARTITIONS)

print(my_graph)


# Step 01: Support Methods to Create CSV files ..

In [ ]:

#  The overall goal of this NoteBook is to show incremental import. That is, importing
#  into the graph many smaller pieces versus perhaps running one single import operation.
#
#  As such, we'll create (n) DataFrames with Nodes, (m) with Edges, and import those.
#
#     (n) and (m) are determined by how we invoke the method below.
#

import pandas as pd

def f_create_dataframe(i_node_or_edge, i_keyranges, i_label_or_type):

   l_data_array = []

   if (i_node_or_edge == "node"):
      #
      #  This is for nodes; we loop from i_keyranges[0] to i_keyranges[1]
      #
      for l_key in range(i_keyranges[0], i_keyranges[1] + 1):
         l_key_str = str(i_label_or_type + "_id").lower()
            #
         l_data_dict = { l_key_str: l_key, "col2": "222", "col3": "333", "col4": 444, "LABEL": i_label_or_type}
            #
         l_data_array.append(l_data_dict)
            #
      l_dataframe =  pd.DataFrame(l_data_array, columns = [l_key_str, "col2", "col3", "col4", "LABEL"])
   else:
      #
      #  This is for edges; here we expect five integers in i_keyranges[]
      #
      #     .  We (outer) loop from i_keyranges[0] to i_keyranges[1]
      #        This is for the left edge, and form the key range for a given
      #        set of nodes.
      #     .  For each record above, we loop (n) times, where n is equal
      #        to i_keyanges[2]. Call this the (inner) loop.
      #     .  The values produced in the inner loop are sequential from
      #        i_keyranges[3] to i_keyranges[4]
      #
      #     .  See examples in next cell
      #
      l_right_key = i_keyranges[3] - 1
         #
      for l_left_key in range(i_keyranges[0], i_keyranges[1] + 1):
         for l_each in range(1, i_keyranges[2] + 1):
            l_right_key +=1
               #
            if (l_right_key > i_keyranges[4]):
               l_right_key = i_keyranges[3]
                  #
            l_data_dict1 = { "start_id": l_left_key,  "end_id": l_right_key, "weight": 444, "uniq_col": str(l_each), "TYPE": i_label_or_type}
            l_data_dict2 = { "end_id": l_left_key,  "start_id": l_right_key, "weight": 444, "uniq_col": str(l_each), "TYPE": i_label_or_type}
               #
            l_data_array.append(l_data_dict1)
            l_data_array.append(l_data_dict2)
            #
         l_dataframe =  pd.DataFrame(l_data_array, columns = ["start_id", "end_id", "weight", "uniq_col", "TYPE"])


   return l_dataframe


print("--")
                             


In [ ]:

#  Example demonstrating the above-
#

#  Create Person nodes in the range 100-103
#
df_nodes1 = f_create_dataframe("node", [100, 103], "Person")
   #                         
print(tabulate(df_nodes1, headers='keys', tablefmt='psql'))


#  Create Flight nodes in the range 400-410
#
df_nodes2 = f_create_dataframe("node", [400, 410], "Flight")
   #
print(tabulate(df_nodes2, headers='keys', tablefmt='psql'))


#  From this method invocation,
#
#     df_edges  = f_create_dataframe("edge", [100, 103, 2, 400, 410], "TAKES_FLIGHT")
#
#  Create 2 TAKES_FLIGHT edges, for each left edge node in the
#  range 100-103. Whatever this loop creates, assign right-edge
#  key value sequentially from 400 to 410.
#
#  If we need more right-edge records, start over at 400.
#
#  If there are not enough left-edge records produced, not
#  all right-edge nodes may recieve an edge.
#
#     If so, you may call this method again, say with these
#     arguments,
#        df_edges  = f_create_dataframe("edge", [100, 103, 1, 404, 406, "TAKES_FLIGHT")
#        df_edges  = f_create_dataframe("edge", [100, 103, 1, 407, 408, "TAKES_FLIGHT")
#        df_edges  = f_create_dataframe("edge", [100, 103, 1, 409, 410, "TAKES_FLIGHT")
#
#     or whatever
#

df_edges  = f_create_dataframe("edge", [100, 103, 2, 400, 410], "TAKES_FLIGHT")
   #
print(tabulate(df_edges, headers='keys', tablefmt='psql'))


del  df_nodes1
del  df_nodes2
del  df_edges

                               
print("--")

#  Sample output
#
#     +----+-------------+--------+--------+--------+---------+
#     |    |   person_id |   col2 |   col3 |   col4 | LABEL   |
#     |----+-------------+--------+--------+--------+---------|
#     |  0 |         100 |    222 |    333 |    444 | Person  |
#     |  1 |         101 |    222 |    333 |    444 | Person  |
#     |  2 |         102 |    222 |    333 |    444 | Person  |
#     |  3 |         103 |    222 |    333 |    444 | Person  |
#     +----+-------------+--------+--------+--------+---------+
#     +----+-------------+--------+--------+--------+---------+
#     |    |   flight_id |   col2 |   col3 |   col4 | LABEL   |
#     |----+-------------+--------+--------+--------+---------|
#     |  0 |         400 |    222 |    333 |    444 | Flight  |
#     |  1 |         401 |    222 |    333 |    444 | Flight  |
#     |  2 |         402 |    222 |    333 |    444 | Flight  |
#     |  3 |         403 |    222 |    333 |    444 | Flight  |
#     |  4 |         404 |    222 |    333 |    444 | Flight  |
#     |  5 |         405 |    222 |    333 |    444 | Flight  |
#     |  6 |         406 |    222 |    333 |    444 | Flight  |
#     |  7 |         407 |    222 |    333 |    444 | Flight  |
#     |  8 |         408 |    222 |    333 |    444 | Flight  |
#     |  9 |         409 |    222 |    333 |    444 | Flight  |
#     | 10 |         410 |    222 |    333 |    444 | Flight  |
#     +----+-------------+--------+--------+--------+---------+
#     +----+------------+----------+----------+------------+--------------+
#     |    |   start_id |   end_id |   weight |   uniq_col | TYPE         |
#     |----+------------+----------+----------+------------+--------------|
#     |  0 |        100 |      400 |      444 |          1 | TAKES_FLIGHT |
#     |  1 |        400 |      100 |      444 |          1 | TAKES_FLIGHT |
#     |  2 |        100 |      401 |      444 |          2 | TAKES_FLIGHT |
#     |  3 |        401 |      100 |      444 |          2 | TAKES_FLIGHT |
#     |  4 |        101 |      402 |      444 |          1 | TAKES_FLIGHT |
#     |  5 |        402 |      101 |      444 |          1 | TAKES_FLIGHT |
#     |  6 |        101 |      403 |      444 |          2 | TAKES_FLIGHT |
#     |  7 |        403 |      101 |      444 |          2 | TAKES_FLIGHT |
#     |  8 |        102 |      404 |      444 |          1 | TAKES_FLIGHT |
#     |  9 |        404 |      102 |      444 |          1 | TAKES_FLIGHT |
#     | 10 |        102 |      405 |      444 |          2 | TAKES_FLIGHT |
#     | 11 |        405 |      102 |      444 |          2 | TAKES_FLIGHT |
#     | 12 |        103 |      406 |      444 |          1 | TAKES_FLIGHT |
#     | 13 |        406 |      103 |      444 |          1 | TAKES_FLIGHT |
#     | 14 |        103 |      407 |      444 |          2 | TAKES_FLIGHT |
#     | 15 |        407 |      103 |      444 |          2 | TAKES_FLIGHT |
#     +----+------------+----------+----------+------------+--------------+

                              

#  Read CSVs into Dask DataFrame

In [ ]:

import dask.dataframe as dd
import numpy as np

print("--")


In [ ]:
%%time

#  Just nodes

#  Sample data,
#
#     Sent|counterparty|cntAppr|cntDecl|cntRiskDecl1|amtRiskDecl|cntRiskDecl2|sumRcvd|cntRcvd|cntDist|cntDecl2
#     0|N|N|N|N|XbRqfTcGJL9fxmNp4simTykwKr|qhBymw13PneK5hpntNVwjNwaYklX8P379JZo|Y|416|0|inactive|1941730454454|0|2018-01-24T01:09:18|9.07632e+09|1419493889|1894897856|2676929303.1430|6614868331.8687|8283173087.0037|6020979256.1980|1984359695|5077917542.6596|1923699935|1948138778|4876422147.1566|

df_nodes = dd.read_csv(
   l_nodes,
   delimiter = "|",
   dtype = {
      "Sent"          : np.dtype("O"),                         #  Change these if you wish,  Numpy datatypes
      "counterparty"  : np.dtype("O"),
      "cntAppr"       : np.dtype("O"),
      "cntDecl"       : np.dtype("O"),
      "cntRiskDecl1"  : np.dtype("O"),
      "amtRiskDecl"   : np.dtype("O"),
      "cntRiskDecl2"  : np.dtype("O"),
      "sumRcvd"       : np.dtype("O"),
      "cntRcvd"       : np.dtype("O"),
      "cntDist"       : np.dtype("O"),
      "cntDecl2"      : np.dtype("O"),
      })

print(len(df_nodes))

print("--")

#  Sample output,
#
#     20000000
#     --
#     CPU times: user 3min, sys: 18 s, total: 3min 18s
#     Wall time: 2min 48s


In [ ]:
%%time

#  Just edges

#  Sample data,
#
#     Account1.id|Account2.id|type|date
#     16089173|16089175|FriendConB|2018-10-10T11:20:05

df_edges = dd.read_csv(
    
   #  l_edges,                                                   #  This would be all files, at once, in memory
   l_edges_small,
    
   delimiter = "|",
   dtype = {
      "Account1.id"   : np.dtype("O"),
      "Account2.id"   : np.dtype("O"),
      "type"          : np.dtype("O"),                           #  These next two column names are keywords in most worlds -- bad idea
      "date"          : np.dtype("O"),
      })

print(len(df_edges))

print("--")

#  Sample output,
#
#     200000000
#     --
#     CPU times: user 4min 25s, sys: 31.2 s, total: 4min 56s
#     Wall time: 4min 23s
